# Analyse de données géospatiales avec Python

## Landsat Data Access

https://www.usgs.gov/landsat-missions/landsat-data-access

Good intro to Landsat images, naming convention: https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-data/landsat-data-in-r-geotiff/

## Rasterion quickstart

https://rasterio.readthedocs.io/en/stable/quickstart.html#id2

The following adapted script 'total.py' comes from the rasterio's github page. It takes the 'RGB.byte.tif' image

In [6]:
# from __future__ import division
import numpy as np
import rasterio
import subprocess

# Read raster bands directly to Numpy arrays.
with rasterio.open('data/RGB.byte.tif') as src:
    r, g, b = src.read()

    # Combine arrays using the 'iadd' ufunc. Expecting that the sum will
    # exceed the 8-bit integer range, initialize it as 16-bit. Adding other
    # arrays to it in-place converts those arrays up and preserves the type
    # of the total array.
    total = np.zeros(r.shape, dtype=rasterio.uint16)
    for band in (r, g, b):
        total += band
    total = total // 3

    # Write the product as a raster band to a new 8-bit file. For keyword
    # arguments, we start with the meta attributes of the source file, but
    # then change the band count to 1, set the dtype to uint8, and specify
    # LZW compression.
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open('./data/example-total.tif', 'w', **kwargs) as dst:
        dst.write(total.astype(rasterio.uint8), indexes=1)

Now that we have the missing "example.tif" file we can follow the quickstart

In [7]:
dataset = rasterio.open('./data/example-total.tif')

Rasterio’s open() function takes a path string or path-like object and returns an opened dataset object. The path may point to a file of any supported raster format. Rasterio will open it using the proper GDAL format driver. Dataset objects have some of the same attributes as Python file objects.

In [8]:
dataset.name

'./data/example-total.tif'

In [9]:
dataset.mode

'r'

In [10]:
dataset.closed

False

### Dataset attributes

Properties of the raster data stored in the example GeoTIFF can be accessed through attributes of the opened dataset object. Dataset objects have bands and this example has a band count of 1.

In [11]:
dataset.count

1

A dataset band is an array of values representing the partial distribution of a single variable in 2-dimensional (2D) space. All band arrays of a dataset have the same number of rows and columns. The variable represented by the example dataset’s sole band is Level-1 digital numbers (DN) for the Landsat 8 Operational Land Imager (OLI) band 4 (wavelengths between 640-670 nanometers). These values can be scaled to radiance or reflectance values. The array of DN values is 7731 columns wide and 7871 rows high.

In [12]:
dataset.width

791

In [13]:
dataset.height

718

Some dataset attributes expose the properties of all dataset bands via a tuple of values, one per band. To get a mapping of band indexes to variable data types, apply a dictionary comprehension to the zip() product of a dataset’s DatasetReader.indexes and DatasetReader.dtypes attributes.

In [14]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint8'}

The example file’s sole band contains unsigned 16-bit [8 bits here !!!] integer values. The GeoTIFF format also supports signed integers and floats of different size.

In [ ]:
### Dataset georeferencing

A GIS raster dataset is different from an ordinary image; its elements (or “pixels”) are mapped to regions on the earth’s surface. Every pixels of a dataset is contained within a spatial bounding box.

In [15]:
dataset.bounds

BoundingBox(left=101985.0, bottom=2611485.0, right=339315.0, top=2826915.0)

Our example covers the world from 358485 meters (in this case) to 590415 meters, left to right, and 4028985 meters to 4265115 meters bottom to top. It covers a region 231.93 kilometers wide by 236.13 kilometers high.

The value of `DatasetReader.bounds` attribute is derived from a more fundamental attribute: the dataset’s geospatial transform.

In [16]:
dataset.transform

Affine(300.0379266750948, 0.0, 101985.0,
       0.0, -300.041782729805, 2826915.0)

## Tutoriel sur Rasterio en Python

Ce tutoriel présente l'utilisation de Rasterio, une bibliothèque Python puissante pour travailler avec des données raster géospatiales, notamment dans le traitement d'images satellites. Voici un aperçu des étapes clés pour lire, traiter et visualiser des images satellites à l'aide de Rasterio.

### Lecture des images satellites

La première étape consiste à lire les données d'image dans un objet de dataset Rasterio. Utilisez la fonction open() pour cela :




In [ ]:
import rasterio

with rasterio.open('image.tif') as dataset:
    # Faites quelque chose avec l'objet dataset

Exploration des métadonnées

Une fois l'image ouverte, vous pouvez explorer ses métadonnées à l'aide des propriétés et méthodes de l'objet dataset :

width = dataset.width
height = dataset.height
bounds = dataset.bounds  # Renvoie les coordonnées minimales et maximales

Traitement des images satellites

Après avoir lu l'image, vous pouvez effectuer diverses opérations de traitement :

    Sous-ensembles : Sélectionnez une zone plus petite de l'image.
    Reprojection : Changez le système de référence de coordonnées (CRS) de l'image.
    Resampling : Modifiez la résolution de l'image.



Exemple de sous-ensemble

Pour extraire un sous-ensemble de l'image :

subset_window = rasterio.windows.Window(100, 50, 100, 150)
subset = dataset.read(window=subset_window)

Exemple de reprojection

Pour reprojeter une image :

from rasterio.crs import CRS

dst_crs = CRS.from_epsg(4326)  # WGS84 CRS
reprojected = rasterio.warp.reproject(
    dataset.read(),
    src_crs=dataset.crs,
    dst_crs=dst_crs,
    resampling=rasterio.enums.Resampling.nearest
)

Exemple de resampling

Pour changer la résolution d'une image :

import numpy as np

resampled = np.empty((dataset.count, int(dataset.height / 2), int(dataset.width / 2)), dtype=dataset.dtypes[0])
rasterio.warp.reproject(
    dataset.read(),
    dataset.transform,
    out_shape=(dataset.count, int(dataset.height / 2), int(dataset.width / 2)),
    resampling=rasterio.enums.Resampling.bilinear,
    out=resampled
)

Visualisation des images satellites

Enfin, vous pouvez visualiser les résultats en utilisant des bibliothèques de traçage comme matplotlib :

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))
ax1.imshow(subset[0], cmap='gray')
ax1.set_title('Sous-ensemble de l\'image originale')
ax2.imshow(reprojected[0], cmap='gray')
ax2.set_title('Image reprojetée')
plt.show()